# Request: PWA (Signals) 


#### 1. Database Connections for DOTDEVRHPGSQL01, dotgissql01, DOTDEV55SQL02
####     Tables: CRASHDATA, GISGRID, DATAWAREHOUSE
#### 2. Get all Signal Locations
#### 3. Convert Signal Locations to Geometry 
#### 4. Retrieve Nearest Intersecton from Signal geometry
#### 5. Label Matches as Signalized intersections

In [1]:
from ris import db2  #library designed for SQL database connection and querying
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)
print os.getcwd()
# %load_ext sql 

Notebook run: 2019-08-14 17:08
E:\RIS\Staff Folders\Samuel\Requests\SH\PWA


In [119]:
"dotgissql01".upper()

'DOTGISSQL01'

### Database connections

In [4]:
cdb = db2.PostgresDb('DOTDEVRHPGSQL01', 'CRASHDATA', quiet = True)
gdb = db2.SqlDb('dotgissql01', 'gisgrid', user='GISUSER', db_pass='GISUSER') #Database Connection
sdb = db2.SqlDb('DOTDEV55SQL02', 'DataWarehouse', user='SHostetter', db_pass='shostetter') #Database Connection

User name (CRASHDATA):soge
Password (CRASHDATA)········


### Retrieving Locations of all Signals  

In [7]:
def sig_lon_lat():
    result = gdb.query("""SELECT [Longitude] lon,[Latitude] lat
                    FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                    where NormalizedType != 'Z'
                    and [Longitude] is not null""")
    return [[float(i[0]),float(i[1])] for i in result.data]


In [13]:
op = sig_lon_lat()
sigs=pd.DataFrame(op,columns=['lon','lat'])

#op
#sigs

	Missing SQL Server Native Client 10.0 datetime2 will not be interpreted correctly



### Converts longitudes and latitudes to geometry (optional)

In [ ]:
#Query Version 1

def lonlat_to_geom(*args): #input either list of list [[lon,lat][lon,lat]] or list lon_lat=[lon,lat] or input lon=lon,lat=lat 
    try:
        
        if any(isinstance(item, list) for item in args[0]): 
            print('TRUE')
            count=0
            geom =[None]*len(args[0])
            for i in args[0]:
                #print(i)
                g=cdb.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=i[0],lat=i[1]))
                #print(g)
                geom[count]=g[0][0][0]
                count+=1

        else:
            print(args[0])
            if isinstance(args[0], list):
                g = cdb.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=args[0][0],lat=args[0][1]))
                geom = g[0][0][0]

    except:
    
        lon_lat = list(args)
        #print(lon_lat)
        g = cdb.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=lon_lat[0],lat=lon_lat[1]))
        geom = g[0][0][0]

    return geom



In [ ]:
#Query Version 2

def lonlat_to_geomv2(*args): #input either list of list [[lon,lat][lon,lat]] or list lon_lat=[lon,lat] or input lon=lon,lat=lat 
    try:
        
        if any(isinstance(item, list) for item in args[0]): 
            print('TRUE')
            count=0
            geom =[None]*len(args[0])
            for i in args[0]:
                #print(lon_lat)
                g=cdb.query("""selectst_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263) geom""".format(lon=int(i[0]),lat=int(i[0])))
                
                geom[count]=g[0][0][0]
                count+=1

        else:
            print(args[0])
            if isinstance(args[0], list):
                g = cdb.query("""select st_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263) geom""".format(lon=args[0][0],lat=args[0][1]))
                geom = g[0][0][0]

    except:
    
        lon_lat = list(args)
        #print(lon_lat)
        g = cdb.query("""select st_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263) geom""".format(lon=lon_lat[0],lat=lon_lat[1]))
        geom = g[0][0][0]

    return geom
    

In [22]:
#ltg = lonlat_to_geom(-73.99247600,40.71507200)
#ltg

'0101000020D7080000F3011E94BF192E419579CDE4A4630841'

In [24]:
#ltg = lonlat_to_geom([-73.99247600,40.71507200])
#ltg

[-73.992476, 40.715072]


'0101000020D7080000F3011E94BF192E419579CDE4A4630841'

In [16]:
#takes 20 secs
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
ltg = lonlat_to_geom(op)
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#ltg

2019-08-14 17:34:34
TRUE
2019-08-14 17:34:59


In [20]:
sigs_geom=pd.DataFrame(ltg,columns=['sig_geom']) #dataframe containing signal geometries
sigs_df = sigs.join(sigs_geom) #dataframe containing signal longitudes, latitudes and geometries

In [19]:
#sigs_df

### Obtains nearest intersection from the location of input signal

In [12]:
#Obtains nearest intersection using signal latitude and longitude

def sig_to_itxv1(lon,lat,tol):

    query = """select nodeid, st_distance(geom, st_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263)) as distance,
                            geom as node_geom,
                            st_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263) as sig_geom
               from node where st_dwithin(st_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263),geom, {tol})
                        and is_int = true
               order by st_distance(geom, st_transform(st_geomfromtext(
                            'SRID=4326;POINT({lon} {lat})'), 2263)) ASC limit 1
               """.format(lon=lon, lat=lat, tol=tol)
    
    result = cdb.query(query)
    return pd.DataFrame(result[0], columns=['nodeid','dist','node_geom','sig_geom'])


In [39]:
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
ans=sig_to_itxv1(-73.99247600,40.71507200,100)
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#ans

In [43]:
#Obtains nearest intersection using signal geometry (geom not initially known)

def sig_to_itxv2(lon,lat,tol):
    lon_lat=[lon,lat]
    sgeom = lonlat_to_geom(lon,lat)
    query = """select nodeid, st_distance(geom, '{sgeom}'::geometry) as distance,
                            geom as node_geom,
                            '{sgeom}'::geometry as sig_geom, '{lon_lat}' as lon_lat
               from node where st_dwithin('{sgeom}'::geometry,geom, {tol})
                        and is_int = true
               order by st_distance(geom, '{sgeom}'::geometry) ASC limit 1
               """.format(sgeom=sgeom, tol=tol, lon_lat=lon_lat)

    result = cdb.query(query)
    return pd.DataFrame(result[0], columns=['nodeid','dist','node_geom','sig_geom','lon_lat'])


In [44]:
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
ans=sig_to_itxv2(-73.99247600,40.71507200,100)
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#ans

2019-08-14 17:57:39
2019-08-14 17:57:39


In [103]:
#Obtains nearest intersection using signal geometry (geom initially known)

def sig_to_itxv3(lon,lat,geom,tol):
    lon_lat=[lon,lat]
    sgeom = geom
    query = """select nodeid, st_distance(geom, '{sgeom}'::geometry) as distance,
                            geom as node_geom,
                            '{sgeom}'::geometry as sig_geom, '{lon_lat}' as lon_lat
               from node where st_dwithin('{sgeom}'::geometry,geom, {tol})
                        and is_int = true
               order by st_distance(geom, '{sgeom}'::geometry) ASC limit 1
               """.format(sgeom=sgeom, tol=tol, lon_lat=lon_lat)

    result = cdb.query(query)
    return result[0][0]

### Obtains all nearest intersections from GISGRID signals

In [109]:
#takes 1mins

def all_sigs_to_itxs(sigs, rad):
    signals = {}
  
    for i in range(len(sigs)): 
        
        
        lon=sigs.iloc[i].lon
        lat=sigs.iloc[i].lat
        geom=sigs_df.iloc[i].sig_geom
        tol=rad      
        
        try:      
            x = sig_to_itxv3(lon,lat,geom,tol)
            nodeid = x[0]
            signals[int(nodeid)]=list(x)[1:]
        
        except:
            pass
        
        
    return signals

In [111]:
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
sigs_dict=all_sigs_to_itxs(sigs_df,100)
print datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

2019-08-14 18:22:57
2019-08-14 18:23:29


In [ ]:
#sig_dict